### **Rupesh Bharambe (AI3107)**

# ##image classification using Keras build in generator

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import matplotlib
matplotlib.use("Agg")
# import the necessary packages
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import cv2
import os

In [3]:
train_dir="/content/drive/MyDrive/Datasets/VegetableDataset/train/"
valid_dir="/content/drive/MyDrive/Datasets/VegetableDataset/validation/"
test_dir="/content/drive/MyDrive/Datasets/VegetableDataset/test/"

In [4]:
train_gen=ImageDataGenerator(rescale=1./255)
val_gen=ImageDataGenerator(rescale=1./255)
#test_gen=ImageDataGenerator(rescale=1./255)

In [5]:
train_data=train_gen.flow_from_directory(train_dir,target_size=(224,224),
                                         batch_size=32,class_mode='categorical')
valid_data=val_gen.flow_from_directory(valid_dir,target_size=(224,224),
                                         batch_size=32,class_mode='categorical')


Found 3115 images belonging to 36 classes.
Found 351 images belonging to 36 classes.


In [7]:
import matplotlib.pyplot as plt
import numpy as np
# Retrieve one batch of images and labels
images, labels = next(train_data)
print(images.shape,labels)
# Display the first image in the batch
plt.imshow(images[0])
plt.axis('off')  # Hide axis
# Display label - access the label for the first image and find the index of the max value
plt.title(f"Label: {np.argmax(labels[0])}")
plt.show()

(32, 224, 224, 3) [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [8]:
class_indices = train_data.class_indices
print(class_indices)

{'apple': 0, 'banana': 1, 'beetroot': 2, 'bell pepper': 3, 'cabbage': 4, 'capsicum': 5, 'carrot': 6, 'cauliflower': 7, 'chilli pepper': 8, 'corn': 9, 'cucumber': 10, 'eggplant': 11, 'garlic': 12, 'ginger': 13, 'grapes': 14, 'jalepeno': 15, 'kiwi': 16, 'lemon': 17, 'lettuce': 18, 'mango': 19, 'onion': 20, 'orange': 21, 'paprika': 22, 'pear': 23, 'peas': 24, 'pineapple': 25, 'pomegranate': 26, 'potato': 27, 'raddish': 28, 'soy beans': 29, 'spinach': 30, 'sweetcorn': 31, 'sweetpotato': 32, 'tomato': 33, 'turnip': 34, 'watermelon': 35}


In [9]:
# import the necessary packages
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras import backend as K

model = Sequential()
inputShape = (224, 224, 3)

# if we are using "channels first", update the input shape
if K.image_data_format() == "channels_first":
			inputShape = (3,224,224)

# first set of CONV => RELU => POOL layers
model.add(Conv2D(16, (3, 3), padding="same",
		input_shape=inputShape))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# second set of CONV => RELU => POOL layers
model.add(Conv2D(32, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# third set of CONV => RELU => POOL layers
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Flatten())
model.add(Dense(1024))
model.add(Activation("relu"))

# sigmoid classifier
model.add(Dense(36, activation="softmax"))
#model.add(Activation("softmax"))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:
#Compile the model
print("[INFO] compiling model...")
opt = Adam()
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])

[INFO] compiling model...


In [16]:
import tensorflow as tf

# Define the checkpoint callback
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath='path/model_epoch_{epoch:02d}.h5',  # Save the model with epoch number
    save_freq='epoch',                      # Save at the end of every epoch
    save_weights_only=False,                # Save the entire model (architecture + weights)
    verbose=1                               # Print a message when saving
)

In [17]:
#train the model
# train the network
print("[INFO] training network...")
H = model.fit(train_data,
            validation_data=valid_data,
						steps_per_epoch=100,
						callbacks=[checkpoint],
						epochs=10)


[INFO] training network...


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
  3/100 ━━━━━━━━━━━━━━━━━━━━ 25:22 16s/step - accuracy: 0.0035 - loss: 6.4154    

/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1043: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


 98/100 ━━━━━━━━━━━━━━━━━━━━ 32s 16s/step - accuracy: 0.0560 - loss: 4.0724

...

100/100 ━━━━━━━━━━━━━━━━━━━━ 153s 1s/step - accuracy: 0.9776 - loss: 0.1968 - val_accuracy: 0.9516 - val_loss: 0.3908
Epoch 7/10
 98/100 ━━━━━━━━━━━━━━━━━━━━ 1s 863ms/step - accuracy: 0.9797 - loss: 0.1400
Epoch 7: saving model to path/model_epoch_07.h5


100/100 ━━━━━━━━━━━━━━━━━━━━ 136s 1s/step - accuracy: 0.9797 - loss: 0.1396 - val_accuracy: 0.9601 - val_loss: 0.1885
Epoch 8/10
 98/100 ━━━━━━━━━━━━━━━━━━━━ 1s 857ms/step - accuracy: 0.9875 - loss: 0.0599
Epoch 8: saving model to path/model_epoch_08.h5


100/100 ━━━━━━━━━━━━━━━━━━━━ 102s 1s/step - accuracy: 0.9875 - loss: 0.0601 - val_accuracy: 0.9516 - val_loss: 0.2441
Epoch 9/10
 98/100 ━━━━━━━━━━━━━━━━━━━━ 1s 853ms/step - accuracy: 0.9935 - loss: 0.0345
Epoch 9: saving model to path/model_epoch_09.h5


100/100 ━━━━━━━━━━━━━━━━━━━━ 152s 1s/step - accuracy: 0.9934 - loss: 0.0348 - val_accuracy: 0.9573 - val_loss: 0.2014
Epoch 10/10
 98/100 ━━━━━━━━━━━━━━━━━━━━ 1s 872ms/step - accuracy: 0.9947 - loss: 0.0287
Epoch 10: saving model to path/model_epoch_10.h5


100/100 ━━━━━━━━━━━━━━━━━━━━ 102s 1s/step - accuracy: 0.9946 - loss: 0.0291 - val_accuracy: 0.9573 - val_loss: 0.2436


In [20]:
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import load_img, img_to_array
import numpy as np
model = load_model('/content/path/model_epoch_10.h5')  # Replace with your model's filename

# Load the image with the same target size as used during training
img = load_img('/content/drive/MyDrive/Datasets/VegetableDataset/test/beetroot/Image_1.jpg', target_size=(224,224))  # Replace with your image path and target size

# Convert the image to a NumPy array
img_array = img_to_array(img)

# Normalize the image data if the model was trained on normalized data
img_array = img_array / 255.0  # Apply this step only if normalization was used during training

# Expand dimensions to match the model's input shape (1, height, width, channels)
img_array = np.expand_dims(img_array, axis=0)
predictions = model.predict(img_array)
predicted_class_index = np.argmax(predictions, axis=1)[0] # Get the numerical class index
print(predicted_class_index) ## numerically

# Find the class name corresponding to the predicted index
predicted_class_name = [k for k, v in class_indices.items() if v == predicted_class_index][0]
print(predicted_class_name) ## categorical prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 281ms/step
2
beetroot
